In [1]:
!pip install -U langchain langchain-community langchain-text-splitters \
langchain-chroma langchain-groq chromadb unstructured \
"unstructured[pdf]" sentence-transformers huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 11.2 MB/s eta 0:00:00


In [4]:
!pip install -U langchain

In [7]:
import os

from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
os.environ["GROQ_API_KEY"] = "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"

In [15]:
import requests
url="https://www.cmu.edu/intelligentbusiness/expertise/genai-principles.pdf"
response=requests.get(url)

In [16]:
response

<Response [200]>

In [17]:
# Save the PDF to a local file
with open("Generative-AI-Principles.pdf", "wb") as f:
    f.write(response.content)

In [18]:
loader = UnstructuredFileLoader("Generative-AI-Principles.pdf")

In [19]:
documents = loader.load()
documents

[Document(metadata={'source': 'Generative-AI-Principles.pdf'}, page_content='Karan Singh, Assistant Professor of Operations Research\n\nPrinciples of Generative AI A Technical Introduction\n\nGenerative artificial intelligence (GenAI) tools are an emerging class of new-age artificial intelligence algorithms capable of producing novel content — in varied formats such as text, audio, video, pictures, and code — based on user prompts. Recent advances in machine learning (ML), massive datasets, and substantial increases in computing power have propelled such tools to human-level performance on academic and professional benchmarks , 1 comparable to the ninetieth percentile on the SAT and the bar exam.\n\nThis rapid progress has led many to to believe that the metamorphosis of these technologies from research-grade demos to accessible and easy-to-use production-grade goods and services carries the potential to supercharge business processes and operations while enabling entirely new delivera

In [20]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [21]:
texts = text_splitter.split_documents(documents)

In [22]:
type(texts)

list

In [23]:
len(texts)

15

In [24]:
texts[4]

Document(metadata={'source': 'Generative-AI-Principles.pdf'}, page_content='We address these questions indirectly in the present section by taking a tour of the essential significant developments in machine learning and artificial intelligence that have occurred in the last decade and have fueled the creation of modern large language models.\n\nClassical Machine Learning as Prediction Machines\n\nWe start with the most well-understood subset of machine learning techniques: supervised learning. The central objective in supervised learning is to produce a prediction rule that predicts well on unseen data, given enough labeled examples. For example, consider predicting house prices from the square footage in a given zip code. Instead of creating a hand-crafted prediction rule, the machine learning methodology advocates for choosing a prediction rule from an expressive but non-exhaustive class of rules, such as linear predictors, that provides the best fit on an existing collection of size

In [25]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/toke

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
persist_directory = "vector_db"

In [27]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [28]:
# retriever
retriever = vectordb.as_retriever()

In [40]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [41]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Use ONLY the context below to answer the question.

Context:
{context}

Question:
{question}

Answer:
""")


In [42]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

qa_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
result = qa_chain.invoke("Summarize the document.")
print(result)

The document discusses the principles and workings of Generative AI (GenAI) models, particularly large language models (LLMs). It highlights the importance of understanding how these models function and the principles they operate on.

The document begins by explaining that LLMs are conditional probabilistic models that produce a completion conditioned on the initial set of words, known as the prompt. It emphasizes the significance of the initial prompt in determining the quality and utility of the synthesized text.

The document then addresses several questions about how LLMs function internally, including how output probabilities for the next word are determined and what goes into creating and using a language model. It takes a tour of essential significant developments in machine learning and artificial intelligence that have fueled the creation of modern LLMs.

The document also explores the similarities and differences between LLMs and supervised learning. It highlights the charac